## Part - b

**Choose 3 distinct organizations (Agency-Branch) giving these awards (e.g., Department of Defense-Airforce) , build ONE LDA topic modeling on all abstracts to identify their areas of interest. Remember to use unigrams, bigrams and trigrams for the LDA computation**

In [24]:
import psycopg2
import pandas as pd

In [25]:
threshold = 100

In [26]:
#!pip install gensim
#!pip install pyLDAvis

In [27]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [28]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [29]:
import nltk; nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/amolkale/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [30]:
conn = psycopg2.connect(database="dse203",
                        host="localhost",
                        user="postgres",
                        password="admin",
                        port="5432")
cursor = conn.cursor()

In [31]:
#picking 3 organization (Agency-Branch) names
sql = '''select funder, "Abstract" from (select case
		   when "Branch" is null then "Agency"
		   else "Agency" || ' - ' || "Branch"
		   end as funder, "Abstract"
        FROM award_data) as f_a
        WHERE "Abstract" like '%radiation hardness%' 
         and funder in (select DISTINCT case
                           when "Branch" is null then "Agency"
                           else "Agency" || ' - ' || "Branch"
                           end as funder
                        FROM award_data 
                        WHERE "Branch" is not null and "Agency" is not null limit 4)'''    
    
cursor.execute(sql)
result = cursor.fetchall()
conn.close()

In [32]:
#Printing the 3 organization combinations we picked.
organizations = [t[0] for t in result]
for org in np.unique(organizations):
    print(org)

Department of Defense - Air Force
Department of Defense - Army
Department of Health and Human Services - National Institutes of Health


In [10]:
#Taking the Abstract column which is at 1st index.
data = [t[1] for t in result]

#print sample Abstract texts
#data[:2]

In [11]:
# Remove Emails
data = [re.sub('\S*@\S*\s?', '', str(sent)) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', str(sent)) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", str(sent)) for sent in data]

#print(data[:1])

In [12]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

#print(data_words[:1])

In [13]:
# Define functions for stopwords and lemmatization
def remove_stopwords(texts):
    return [[word for word in doc if word not in stop_words] for doc in texts]


def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [14]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_nostops, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

#print(data_lemmatized[0])

/Users/amolkale/anaconda3/lib/python3.8/site-packages/spacy/language.py:1895: UserWarning: [W123] Argument disable with value ['parser', 'ner'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(


In [15]:
#Creating bigrams and trigrams
bigram = gensim.models.Phrases(data_lemmatized, min_count=3, threshold=threshold)
trigram = gensim.models.Phrases(bigram[data_lemmatized],threshold=threshold)
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [16]:
#bigrams and trigrams routine
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

In [17]:
data_word_trigrams = make_trigrams(data_lemmatized)
id2word_tri = corpora.Dictionary(data_word_trigrams)

In [18]:
# Create Dictionary
#id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_word_trigrams

# Term Frequency list   
corpus = [id2word_tri.doc2bow(text) for text in texts]

In [19]:
for num_topics in range(3, 11, 1):
    
    # Build LDA model
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                               id2word=id2word_tri,
                                               num_topics=num_topics, 
                                               random_state=100,
                                               update_every=1,
                                               chunksize=100,
                                               passes=10,
                                               alpha='auto',
                                               per_word_topics=True)
    # Compute Perplexity
    #print('\nPerplexity: ', lda_model.log_perplexity(corpus))  
    # a measure of how good the model is. lower the better.

    # Compute Coherence Score
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word_tri, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    #print('\nCoherence Score: ', coherence_lda)
    
    print("Num Topics: {0} \t Perplexity Score: {1} \t Coherence Score: {2}".format(num_topics,lda_model.log_perplexity(corpus),coherence_lda))

Num Topics: 3 	 Perplexity Score: -6.896012010235031 	 Coherence Score: 0.3038803137485894
Num Topics: 4 	 Perplexity Score: -6.911085380202709 	 Coherence Score: 0.31077921073265713
Num Topics: 5 	 Perplexity Score: -6.915923565136269 	 Coherence Score: 0.3029136040012011
Num Topics: 6 	 Perplexity Score: -6.893987405542958 	 Coherence Score: 0.3431617162133775
Num Topics: 7 	 Perplexity Score: -6.916733013828991 	 Coherence Score: 0.32031061994732973
Num Topics: 8 	 Perplexity Score: -6.942199095325276 	 Coherence Score: 0.30578820216453806
Num Topics: 9 	 Perplexity Score: -6.961317718721443 	 Coherence Score: 0.30269057728431803
Num Topics: 10 	 Perplexity Score: -6.948563516669897 	 Coherence Score: 0.3196394516587067


Since Coherence for 6 topics highest, let's choose that one.

In [20]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word_tri,
                                           num_topics=6, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [21]:
# Print the Keyword in the 6 topics
print(lda_model.print_topics())

[(0, '0.019*"process" + 0.018*"technology" + 0.015*"radiation" + 0.012*"product" + 0.012*"commercial" + 0.010*"market" + 0.008*"space" + 0.008*"harden" + 0.008*"use" + 0.007*"circuit"'), (1, '0.037*"power" + 0.017*"high" + 0.012*"low" + 0.011*"radiation" + 0.011*"design" + 0.010*"converter" + 0.010*"application" + 0.010*"technology" + 0.009*"system" + 0.008*"phase"'), (2, '0.029*"high" + 0.019*"device" + 0.019*"radiation" + 0.014*"system" + 0.013*"application" + 0.012*"power" + 0.010*"communication" + 0.010*"use" + 0.009*"speed" + 0.009*"performance"'), (3, '0.018*"radiation" + 0.018*"design" + 0.016*"high" + 0.014*"use" + 0.013*"power" + 0.009*"application" + 0.009*"space" + 0.009*"system" + 0.009*"technology" + 0.008*"base"'), (4, '0.026*"radiation" + 0.021*"memory" + 0.020*"harden" + 0.017*"design" + 0.014*"device" + 0.010*"space" + 0.009*"high" + 0.009*"power" + 0.009*"performance" + 0.009*"application"'), (5, '0.033*"cell" + 0.031*"solar" + 0.022*"efficiency" + 0.020*"high" + 0.01

In [22]:
#preparing vizualization
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word_tri)

/Users/amolkale/anaconda3/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


In [23]:
#Displaying the visualization.
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.013668  0.013841       1        1  25.351283
4     -0.054841  0.055892       2        1  19.262005
0     -0.133655 -0.089385       3        1  17.404526
1      0.058283  0.040395       4        1  14.007110
5      0.112343 -0.104845       5        1  12.887606
2      0.004202  0.084104       6        1  11.087470, topic_info=           Term        Freq       Total Category  logprob  loglift
430       solar   54.000000   54.000000  Default  30.0000  30.0000
398        cell   87.000000   87.000000  Default  29.0000  29.0000
16   efficiency   57.000000   57.000000  Default  28.0000  28.0000
36        power  169.000000  169.000000  Default  27.0000  27.0000
30       memory   69.000000   69.000000  Default  26.0000  26.0000
..          ...         ...         ...      ...      ...      ...
386       space    8.608610   99.419101   Topic6  -5.0936  -0.2472
188  technology    9.025489  124.309813   Topic6  -5.0463  -0.4234
134    material    7.024645   45.593497   Topic6  -5.2969   0.3290
14       design    6.725008  147.056714   Topic6  -5.3405  -0.8856
443     benefit    6.023277   57.320769   Topic6  -5.4507  -0.0537

[432 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
1075      5  0.718296     absorption
2179      5  0.796552  accelerometer
63        1  0.168805        achieve
63        2  0.042201        achieve
63        3  0.084403        achieve
...     ...       ...            ...
1853      6  0.623516       waveform
1656      3  0.882489          width
1289      2  0.771825           word
1289      4  0.192956           word
1892      1  0.908684            zno

[753 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 5, 1, 2, 6, 3])

/Users/amolkale/anaconda3/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/amolkale/anaconda3/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/amolkale/anaconda3/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/amolkale/anaconda3/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/amolkale/anaconda3/lib/python3.8/site-packages/past/builtins/misc.py:45: 

From the visualization , we can see that our current configuration gives minimum overlap of topic bubbles. So we are confident about our selection.